In [24]:
import numpy as np
import pandas as pd
import random
import copy

In [25]:
matrix_km = pd.read_excel('data/df_distance_km.xlsx')
matrix_km.index = matrix_km.columns

class Vehicle():
    def __init__(self, id, capacidad_kg, costo_km, autonomia_km):
        self.id = id
        self.capacidad_kg = capacidad_kg
        self.costo_km = costo_km
        self.autonomia_km = autonomia_km
    def get_price(self, distance):
        return round(self.costo_km * distance, 2)

vehicles = []
for _, row in pd.read_excel('data/df_vehicle.xlsx').iterrows():
    vehicles.append(Vehicle(
        id = int(row['vehiculo_id']), 
        capacidad_kg= float(row['capacidad_kg']),
        costo_km= float(row['costo_km']), 
        autonomia_km= int(row['autonomia_km'])
        ))
vehiculos_2 = vehicles.copy()

class Client():
    def __init__(self, cliente, order_kg):
        self.cliente = cliente
        self.order_kg = order_kg

class Order():
    def __init__(self, clients, vehicle, route):
        self.clients = clients
        self.vehicle = vehicle
        self.route = route

    def get_clients(self):
        return self.clients
    
    def get_kg(self):
        return [i.order_kg for i in self.clients]
    
    def get_route(self):
        final_route = []
        for i in self.route:
            if i == 'Almacén':
                final_route.append([i, 0])
            for client in self.clients:
                if client.cliente == i:
                    final_route.append([i, client.order_kg])
        return final_route

In [26]:
def check_viability(ruta):
    for i in range(0,len(ruta),1):
        try:
            if (matrix_km[ruta[i]].client, ruta[i+1].cliente != 0) == False:
                return False
        except:
            pass
    return True

def create_route(clients):
    random.shuffle(clients)
    route = clients
    while check_viability(route) == False:
        random.shuffle(clients)
        ['Almacén'] + clients + ['Almacén']
    return route

In [27]:
clientes = []
df_demandas = pd.read_excel('data/df_historic_order_demand.xlsx')
df_demandas[df_demandas['mes_anio'] == '12-2020']
for cliente, order_demand in zip(df_demandas[df_demandas['mes_anio'] == '12-2020']['cliente'], df_demandas[df_demandas['mes_anio'] == '12-2020']['order_demand']):
    clientes.append(Client(cliente, order_demand)) 


In [28]:
p = list(df_demandas[df_demandas['mes_anio'] == '12-2020']['cliente'])
ruta = ['Almacén'] + create_route(p) + ['Almacén']
order1 = Order(clientes, vehicles[1], ruta)
order1.get_route()

[['Almacén', 0],
 ['Cliente_15', 891.0],
 ['Cliente_11', 861.0],
 ['Cliente_20', 923.0],
 ['Cliente_5', 932.0],
 ['Cliente_1', 857.0],
 ['Cliente_18', 887.0],
 ['Cliente_8', 942.0],
 ['Cliente_19', 852.0],
 ['Cliente_10', 923.0],
 ['Cliente_7', 895.0],
 ['Cliente_9', 856.0],
 ['Cliente_13', 944.0],
 ['Cliente_12', 926.0],
 ['Cliente_2', 941.0],
 ['Cliente_14', 904.0],
 ['Cliente_3', 878.0],
 ['Cliente_16', 942.0],
 ['Cliente_6', 873.0],
 ['Cliente_17', 906.0],
 ['Cliente_4', 949.0],
 ['Almacén', 0]]

In [29]:
def generate_initial_solution(vehicles, route):
    solucion = {}
    clients = [i.cliente for i in route]
    random.shuffle(clients)

    vh_copy = vehicles.copy()
    random.shuffle(vh_copy)

    route_vehicles = vh_copy

    ruta = ['Almacén'] + [i.cliente for i in route] + ['Almacén']
    counter = 0
    while clients:
        bucle = True
        vehiculo = random.choice(route_vehicles)

        solucion[vehiculo.id] = {
            'clientes' : [],
            'distancia_total' : 0,
            'peso_total' : 0,
            'costo_total' : 0
        }

        #print('\n\nVehículo escogído: ',vehiculo.id, ' capacidad: ',vehiculo.capacidad_kg, ' autonomía: ', vehiculo.autonomia_km)
        carga_actual = 0
        distancia_actual = 0
        while clients and bucle:
            temp_route = ruta[counter:counter+2]
            distance = matrix_km[temp_route[0]][temp_route[1]]
            distance_to_warehouse = matrix_km[temp_route[0]]['Almacén']
            load = [i.order_kg for i in clientes if i.cliente == temp_route[1]][0]

            if (carga_actual+load)<=vehiculo.capacidad_kg and (distancia_actual+distance+distance_to_warehouse)<=vehiculo.autonomia_km:
                #print(clients)
                if distancia_actual == 0:
                    distancia_actual += (distance +distance_to_warehouse)
                else:
                    distancia_actual += distance
                #print(f'CARGA: {temp_route[1]} --> {[i.order_kg for i in clientes if i.cliente == temp_route[1]][0]}')
                carga_actual += load
                #print(f'Carga actual: {carga_actual}, Distancia actual: {distancia_actual}')
                #print (temp_route[0], ' --> ', temp_route[1],' ---> ', matrix_km[temp_route[0]][temp_route[1]],'km ',f'carga_actual: {carga_actual}', f'distancia_actual: {distancia_actual} \n')
                del clients[0]
                solucion[vehiculo.id]['clientes'].append(temp_route[1])
                counter += 1
            else:
                solucion[vehiculo.id]['distancia_total'] += round((distancia_actual+distance_to_warehouse),2)
                solucion[vehiculo.id]['peso_total'] += carga_actual
                solucion[vehiculo.id]['costo_total'] += round(vehiculo.costo_km * (distancia_actual+distance_to_warehouse),2)
                
                route_vehicles.remove(vehiculo)
                #print('\n ¡Has excedido la carga! \n ')
                bucle = False
        if len(clients) < 1:
            ultima_parada = ruta[-2]
            distancia_regreso = matrix_km[ultima_parada]['Almacén']
            solucion[vehiculo.id]['distancia_total'] += round(distancia_actual,2)
            solucion[vehiculo.id]['peso_total'] += carga_actual
            solucion[vehiculo.id]['costo_total'] += round(vehiculo.costo_km * distancia_actual,2)
            if distancia_regreso<=vehiculo.autonomia_km:
                distancia_actual += distancia_regreso
                #print(f'\nRegresando al Almacén desde {ultima_parada} ---> {distancia_regreso} km')
            else:
                route_vehicles.remove(vehiculo)
                #print('\n ¡Has excedido la carga! \n ')
    return(solucion)


route = generate_initial_solution(vehicles=vehicles, route=create_route(clientes))

In [30]:
def fitness(solution):
    total_cost = 0
    for vehicle_id, details in solution.items():
        total_cost += details['costo_total']  # Sumar el costo total de cada vehículo
    return -total_cost

def create_population(clients, size):
    population = []
    for _ in range(size):
        route = create_route(clients)
        population.append(route)
    return population

population = create_population(clientes, 100)
results = [fitness(generate_initial_solution(vehicles, i)) for i in population ]
results


[np.float64(-52.92),
 np.float64(-60.39),
 np.float64(-53.73),
 np.float64(-54.64),
 np.float64(-51.14),
 np.float64(-58.06000000000001),
 np.float64(-67.44),
 np.float64(-59.410000000000004),
 np.float64(-52.31),
 np.float64(-65.14999999999999),
 np.float64(-56.49),
 np.float64(-54.629999999999995),
 np.float64(-54.43),
 np.float64(-59.77),
 np.float64(-54.52),
 np.float64(-52.92),
 np.float64(-56.540000000000006),
 np.float64(-58.84000000000001),
 np.float64(-49.870000000000005),
 np.float64(-68.32000000000001),
 np.float64(-61.36),
 np.float64(-68.27),
 np.float64(-54.3),
 np.float64(-56.97),
 np.float64(-60.74),
 np.float64(-59.31),
 np.float64(-59.660000000000004),
 np.float64(-52.09),
 np.float64(-58.16),
 np.float64(-51.14),
 np.float64(-61.239999999999995),
 np.float64(-55.95),
 np.float64(-58.06000000000001),
 np.float64(-69.86),
 np.float64(-54.260000000000005),
 np.float64(-57.45),
 np.float64(-57.839999999999996),
 np.float64(-47.28),
 np.float64(-56.4),
 np.float64(-60.39)

In [31]:
def selection(population, results, tournament_size=5):
    tournament = random.sample(list(zip(population, results)), tournament_size)
    tournament.sort(key=lambda x: x[1], reverse=True)
    return [i[0] for i in tournament[0:2]]

def crossover(parents):
    # Une la mitad del padre 1 y la otra mitad del padre 2 mientras el cliente no esté en padre1
    parent1, parent2 = parents
    midpoint = len(parent1) // 2
    child = parent1[:midpoint] + [client for client in parent2 if client not in parent1[:midpoint]]
    return child

def mutate(route, mutation_rate=0.1):
    # Coge una ruta y si entra dentro del 
    if  random.random() < mutation_rate:
        i, j = random.sample(range(0,len(route)), 2)
        route[i], route[j] = route[j], route[i]  # Intercambiar dos clientes
    return route

best_routes = selection(population, results, tournament_size=5)
new_route = mutate(crossover(best_routes))
len(new_route)


20

In [35]:
def genetic_algorithm(clients, vehicles, matrix_km, population_size=100, generations=50, mutation_rate=0.1):
    # Generar población inicial con sus resultados
    population = create_population(clients, population_size)
    results = [fitness(generate_initial_solution(vehicles, i)) for i in population ]  # Calcular fitness inicial
    routes = []

    for generation in range(generations):
        new_population = []
        new_results = []
        new_routes = []
        
        for _ in range(population_size):
            parents = selection(population, results)
            child = crossover(parents)
            child = mutate(child, mutation_rate)
            new_population.append(child)
        
        for j in new_population:
            route = generate_initial_solution(vehicles, j)
            route_value = fitness(route)

            new_results.append(route_value)
            new_routes.append(route)
        
        population = new_population
        results = new_results
        routes = new_routes

        # Mejor solución en la generación actual
        best_cost = min(results)
        print(f"Generación {generation}: Mejor costo: {-best_cost}")  # Invertir para mostrar el costo original
    
    # Mejor solución final
    best_index = results.index(max(results))  # Seleccionar el mejor fitness
    
    return population[best_index], -results[best_index], routes[best_index]  # Retornar ruta y costo
best_population, best_cost, route = genetic_algorithm(clientes, vehicles, matrix_km, population_size=100, generations=200, mutation_rate=0.1)
print('Mejor población: ', [i.cliente for i in best_population])
print("Costo de la mejor ruta:", best_cost)
print('Ruta final: ')
for i,k in route.items():
    print(
        f"\nVehículo = {i} \n Clientes: {k['clientes']}\n Distancia Total: {k['distancia_total']}\n Peso Total: {k['peso_total']}\n Costo Total: {k['costo_total']}"
    )


Generación 0: Mejor costo: 79.03000000000002
Generación 1: Mejor costo: 82.51
Generación 2: Mejor costo: 84.41999999999999
Generación 3: Mejor costo: 85.78
Generación 4: Mejor costo: 78.37
Generación 5: Mejor costo: 80.63000000000001
Generación 6: Mejor costo: 82.60000000000001
Generación 7: Mejor costo: 72.39999999999999
Generación 8: Mejor costo: 72.43
Generación 9: Mejor costo: 72.31
Generación 10: Mejor costo: 70.05
Generación 11: Mejor costo: 66.83
Generación 12: Mejor costo: 66.25
Generación 13: Mejor costo: 69.98
Generación 14: Mejor costo: 68.27000000000001
Generación 15: Mejor costo: 63.46
Generación 16: Mejor costo: 62.61
Generación 17: Mejor costo: 67.44
Generación 18: Mejor costo: 67.0
Generación 19: Mejor costo: 66.49
Generación 20: Mejor costo: 67.24000000000001
Generación 21: Mejor costo: 66.03
Generación 22: Mejor costo: 60.0
Generación 23: Mejor costo: 62.410000000000004
Generación 24: Mejor costo: 57.67
Generación 25: Mejor costo: 60.68
Generación 26: Mejor costo: 58.